In [9]:
import os
import pandas as pd
import mygene
import statistics
import glob
os.chdir("../../data/recipients/")

In [ ]:
def ensembl2gene():
    ensmeble2gene={}
    fx=pd.read_csv("wgcna/gene2ensemble_fx.csv")
    hx=pd.read_csv("wgcna/gene2ensemble_hx.csv")
    temp=pd.concat([fx,hx])
    genes={}
    for i,row in temp.iterrows():
        genes[row["ensbl"]]=row["gene"]
    
    return genes


def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [10]:
edges=pd.read_csv("mmvec.edge.recipients.txt",sep="\t")
nodes=pd.read_csv("wKDA_reanalysis/q6JZ4or4TB.wKDA_cytoscape_nodes.txt",sep="\t")
kds=pd.read_csv("wKDA_reanalysis/q6JZ4or4TB.wKDA_kd_full_results.txt",sep="\t")

In [11]:
degs_hx=[]
degs_fx=[]
for i,row in nodes[nodes['MODULE'].notna()].iterrows():
    module=row["MODULE"]
    node=row["NODE"]
    if("DEG_fx" in module):
        degs_hx.append(node.replace("_hx",""))
    elif("DEG_hx" in module):
        degs_fx.append(node.replace("_fx",""))

In [12]:
en2gene=ensembl2gene()
degs_hx=[]
for i in en2gene.keys():
    degs_hx.append(en2gene[i]+"_hx")

en2gene=ensembl2gene()
degs_fx=[]
for i in en2gene.keys():
    degs_fx.append(en2gene[i]+"_fx")

In [ ]:
gwasoverlap=pd.read_csv("wgcna/GWAS.overlap.txt",sep="\t")
gwas_overlap=list(gwasoverlap["MGI.symbol"])
gwas_overlap=[]
for gene in gwasoverlap["MGI.symbol"]:
    gwas_overlap.append(gene+"_hx")

In [ ]:
genes=ensembl2gene()

for i,row in edges.iterrows():
    if "hx" in row["HEAD"]:
        gene=row["HEAD"].split("_")[0]
        if(gene in genes.keys()):
            edges.iloc[i,0]=genes[gene]+"_hx"
    elif "fx" in row["HEAD"]:
        gene=row["HEAD"].split("_")[0]
        if(gene in genes.keys()):
            edges.iloc[i,0]=genes[gene]+"_fx"
            
    if "hx" in row["TAIL"]:
        gene=row["TAIL"].split("_")[0]
        if(gene in genes.keys()):
            edges.iloc[i,1]=genes[gene]+"_hx"
    elif "fx" in row["TAIL"]:
        gene=row["TAIL"].split("_")[0]
        if(gene in genes.keys()):
            edges.iloc[i,1]=genes[gene]+"_fx"

for i,row in nodes.iterrows():
    if "hx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            nodes.iloc[i,0]=genes[gene]+"_hx"
            nodes.iloc[i,1]=genes[gene]+"_hx"
    if "fx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            nodes.iloc[i,0]=genes[gene]+"_fx"
            nodes.iloc[i,1]=genes[gene]+"_fx"

for i,row in kds.iterrows(): 
    if "hx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            kds.iloc[i,1]=genes[gene]+"_hx"
    if "fx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            kds.iloc[i,1]=genes[gene]+"_fx"

In [ ]:
nodes["GWAS"]=""
nodes["DEGS"]=""

In [ ]:
selected_kds=["DTDPRHAMSYN-PWY: dTDP-L-rhamnose biosynthesis I|g__Bacteroides.s__Bacteroides_cellulosilyticus",
             "Atf4_hx","PWY-5101: L-isoleucine biosynthesis II|g__Lachnoclostridium.s__Clostridium_hylemonae",
             "PWY-6507: 4-deoxy-L-threo-hex-4-enopyranuronate degradation|g__Lachnoclostridium.s__Clostridium_aldenense",
             "Olfm1_hx",
             "Gm15206_fx",
             "Zfp385b_hx",
             "Tor1b_hx",
             "Gm37848_hx",
              "Lcmt2_hx",
              "Stra6_fx",
              "PWY-7446: sulfoglycolysis|g__Citrobacter.s__Citrobacter_amalonaticus",
              "Cxcl10_hx",
              "Mrps2_hx",
              "Loxl3_hx",
              "Rassf6_hx",
              "PWY-1861: formaldehyde assimilation II (RuMP Cycle)|g__Lachnoclostridium.s__Clostridium_aldenense",
              "PWY-6609: adenine and adenosine salvage III|g__Bacteroides.s__Bacteroides_sartorii",
              "VALSYN-PWY: L-valine biosynthesis|g__Bacteroides.s__Bacteroides_sartorii",
              "PWY-6703: preQ0 biosynthesis|g__Alistipes.s__Alistipes_putredinis",
              "PWY-3841: folate transformations II|g__Bacteroides.s__Bacteroides_xylanisolvens"
             ]

In [ ]:
node2module=pd.read_csv("wgcna/NODES4wKDA.tsv",sep="\t")
node2module=node2module.set_index("NODE")

In [ ]:
modulepath={"hx":"../../modules/modules-hx/gene_name/",
           "fx":"../../modules/modules-fx/gene_name/",
           "serum":"../../modules/modules-serum/modules/",
           "feces":"../../modules/modules-feces/modules/",
           "meta":"../../modules/metagenome_modules/"}

In [ ]:
genes=ensembl2gene()

for i,row in node2module.iterrows():
    if "hx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            node2module.iloc[i,1]=genes[gene]+"_hx"
    if "fx" in row["NODE"]:
        gene=row["NODE"].split("_")[0]
        if(gene in genes.keys()):
            node2module.iloc[i,1]=genes[gene]+"_fx"
for tissue in modulepath.keys():
    mp=modulepath[tissue]
    module_files=glob.glob(mp+"/*.txt")
    for mf in module_files:
        md=os.path.basename(mf).replace(".txt","").replace(".new","")
        with open(mf,"r") as f:
            for line in f:
                if(tissue =="meta"):
                    node2module=node2module.append({"MODULE":md+"_"+tissue,"NODE":line.strip()},ignore_index = True)
                    #node2module[line.strip()]=md+"_"+tissue
                elif(tissue=="fx" or tissue=="hx"):
                    node2module=node2module.append({"MODULE":md+"_"+tissue,"NODE":line.strip().capitalize()+"_"+tissue},ignore_index = True)
                else:
                    node2module=node2module.append({"MODULE":md+"_"+tissue,"NODE":line.strip()+"_"+tissue},ignore_index = True)
                    #node2module[line.strip()+"_"+tissue]=md+"_"+tissue
node2module=node2module.drop_duplicates()
node2module=node2module.reset_index()

In [ ]:
deg_fx=pd.read_csv("wgcna/DEGs/Degs_frontal_cortex.txt",sep="\t")+"_fx"
deg_hx=pd.read_csv("wgcna/DEGs/Degs_hippo.txt",sep="\t")+"_hx"
deg_feces=pd.read_csv("wgcna/DEGs/degs_feces.txt",sep="\t")
deg_serum=pd.read_csv("wgcna/DEGs/degs_serum.txt",sep="\t")
deg_fx.columns=["NODE"]
deg_hx.columns=["NODE"]
deg=pd.concat([deg_fx,deg_hx,deg_feces,deg_serum])

In [ ]:
node2module=node2module.set_index("MODULE")
deg_gwas_hx=list(node2module.loc["GWAS_hx","NODE"])
node2module=node2module.reset_index()
node2module=node2module.set_index("NODE")
kds=kds.reset_index()
kds=kds.set_index("NODE")
node2module=node2module.reset_index()
node2module=node2module.set_index("MODULE")
node2module=node2module.loc[list(set(node2module.index)-set(["DEG_fx","DEG_hx","DEG_serum","DEG_feces","GWAS_hx"])),:]
node2module=node2module.reset_index()
node2module=node2module.set_index("NODE")
nodes=nodes.set_index("NODE")

In [ ]:
module4kd_count={}
for kd in selected_kds:
    moduleFromKDS=[]
    moduleFromnode2module=[]
    if(isinstance(kds.loc[kd,"MODULE"], pd.core.series.Series)):
        tmplist=[]
        for md in kds.loc[kd,"MODULE"].tolist():
            tmplist.append(md+"_"+kd)
        moduleFromKDS=tmplist
        #moduleFromKDS=kds.loc[kd,"MODULE"].tolist()
    else:
        moduleFromKDS=[kds.loc[kd,"MODULE"]+"_"+kd]
    if(isinstance(node2module.loc[kd,"MODULE"], pd.core.series.Series)):
        tmplist=[]
        for md in node2module.loc[kd,"MODULE"].tolist():
            tmplist.append(md+"_"+kd)
        moduleFromKDS=tmplist
        #moduleFromnode2module=node2module.loc[kd,"MODULE"].tolist()
    else:
        moduleFromnode2module=[node2module.loc[kd,"MODULE"]+"_"+kd]
    modules4kd=set(moduleFromKDS+ moduleFromnode2module)
    for md in modules4kd:
        module4kd_count[md]=0

In [ ]:
kd_list=[]
module4kd_count={}
for kd in selected_kds:
    moduleFromKDS=[]
    moduleFromnode2module=[]
    if(isinstance(kds.loc[kd,"MODULE"], pd.core.series.Series)):
        tmplist=[]
        for md in kds.loc[kd,"MODULE"].tolist():
            tmplist.append(md+"_"+kd)
        moduleFromKDS=tmplist
        #moduleFromKDS=kds.loc[kd,"MODULE"].tolist()
    else:
        moduleFromKDS=[kds.loc[kd,"MODULE"]+"_"+kd]
    if(isinstance(node2module.loc[kd,"MODULE"], pd.core.series.Series)):
        tmplist=[]
        for md in node2module.loc[kd,"MODULE"].tolist():
            tmplist.append(md+"_"+kd)
        moduleFromKDS=tmplist
        #moduleFromnode2module=node2module.loc[kd,"MODULE"].tolist()
    else:
        moduleFromnode2module=[node2module.loc[kd,"MODULE"]+"_"+kd]
    modules4kd=set(moduleFromKDS+ moduleFromnode2module)
    for md in modules4kd:
        module4kd_count[md]=0

for kd in selected_kds:
    subset=edges.loc[edges["HEAD"]==kd,:]
    kd_on_tail=False
    if len(subset) ==0:
        subset=edges.loc[edges["TAIL"]==kd,:]
        kd_on_tail=True
    for i,row in subset.iterrows():
        target=row["TAIL"]
        if(row["TAIL"]==row["HEAD"]):
            continue
        if(kd_on_tail):
            target=row["HEAD"]
#         if "_hx" in target or "_fx" in target:
#             print(target)
            #target=target.capitalize()
        node_module=node2module[node2module.index==target]["MODULE"]
        #node_module=node2module.loc[target,"MODULE"]
        #if in deg list and module
        #SCAN For DEGS and GWAS first
        if(isinstance(node_module, pd.core.series.Series)):
            for nm in node_module:
                nm=nm+"_"+kd
#                 if(nm not in module4kd_count.keys()):
#                     module4kd_count[nm]=0
                if(nm in module4kd_count.keys()):
                    if(target.casefold() in (node.casefold() for node in deg["NODE"])):
                        kd_list.append(row.to_frame().transpose())
                        module4kd_count[nm]=module4kd_count[nm]+1
                        nodes.loc[target,"DEGS"]="T"
                    if(target in deg_gwas_hx):
                        kd_list.append(row.to_frame().transpose())
                        module4kd_count[nm]=module4kd_count[nm]+1
                        nodes.loc[target,"GWAS"]="T"
                    if("_serum" in target):
                        print(target)
                        kd_list.append(row.to_frame().transpose())
                        module4kd_count[nm]=module4kd_count[nm]+1
                    
        else:
            node_module=node_module+"_"+kd
#             if(node_module not in module4kd_count.keys()):
#                 module4kd_count[node_module]=0
            if(node_module in module4kd_count.keys()):
                if(target.casefold() in (node.casefold() for node in deg["NODE"])):
                    kd_list.append(row.to_frame().transpose())
                    module4kd_count[node_module]=module4kd_count[node_module]+1
                    nodes.loc[target,"DEGS"]="T"
                if(target in deg_gwas_hx):
                    kd_list.append(row.to_frame().transpose())
                    module4kd_count[node_module]=module4kd_count[node_module]+1
                    nodes.loc[target,"GWAS"]="T"
                if("_serum" in target):
                        print(target)
                        kd_list.append(row.to_frame().transpose())
                        module4kd_count[nm]=module4kd_count[nm]+1
                    
    #Fill rest of the network connected to KD 
    for i,row in subset.iterrows():
        target=row["TAIL"]
        if(row["TAIL"]==row["HEAD"]):
            continue
        if(kd_on_tail):
            target=row["HEAD"]
        if "_hx" in target or "_fx" in target:
            target=target.capitalize()
        node_module=node2module[node2module.index==target]["MODULE"]
        #node_module=node2module.loc[target,"MODULE"]
        #if in deg list and module
        if(isinstance(node_module, pd.core.series.Series)):
            for nm in node_module:
                nm=nm+"_"+kd
                if(nm in module4kd_count.keys()):
                    if module4kd_count[nm] < 10:
                        kd_list.append(row.to_frame().transpose())
                        module4kd_count[nm]=module4kd_count[nm]+1
        else:
            node_module=node_module+"_"+kd
            if(node_module in module4kd_count.keys()):
                if module4kd_count[node_module] < 10:
                    kd_list.append(row.to_frame().transpose())
                    module4kd_count[node_module]=module4kd_count[node_module]+1
                    
nodes.to_csv("newnodes.txt",sep="\t",index=False)
selected_edges=pd.concat(kd_list).reset_index(drop=True).drop_duplicates()
selected_edges.to_csv("selected_edges.txt",sep="\t",index=False)